In [1]:
import pandas
import re
import numpy as np

In [2]:
data = pandas.read_csv("climbs2.csv")
len(data.index)

110974

In [271]:
#### clean up data ####

In [3]:
data.name = data.name.map(lambda x: x.lower())
data.area = data.area.map(lambda x: x.lower())
data.region = data.region.map(lambda x: x.lower())
data.crag = data.crag.map(lambda x: x.lower())

In [4]:
# remove all climbs with "unknown" name (there are 365 of them)
data = data[data.name != "unknown"]

In [5]:
# remove international climbs 
data = data[data.region != "international"]
len(data.index)

98319

In [6]:
#clean up urls 
def url_cleanup(url):
    match = re.search('/v/.+/\d+', url)
    if match:
        new_url = 'http://mountainproject.com' + match.group()
    return new_url

In [7]:
data.url = data.url.apply(url_cleanup)

In [8]:
len(set(data.url)) #turns out there are only 63305 unique urls! - remove the duplicates

63305

In [9]:
#drop duplicate values
data = data.drop_duplicates('url')
len(data.index)

63305

In [10]:
# remove rows with incorrect urls (because they lead to duplicates)
def remove_invalid_urls(url):
    match = re.search('/v/[^/]+/\d+', url)
    if match:
        goodrow = 1
    else:
        goodrow = np.NaN
    
    return goodrow

In [11]:
keep = data.url.apply(remove_invalid_urls)

In [12]:
data = data.ix[keep == 1, :]

In [13]:
len(data.index)

62969

In [14]:
# if there are still climbs with same name and area, get rid of them

In [15]:
data = data.drop_duplicates(['name', 'area'])

In [16]:
len(data.index) #keeping 62923 climbs in database
data.isnull().sum()

name                0
area                0
climb_info          0
crag                0
region              0
description         0
number_of_votes     0
grade               0
protection         12
url                 0
areaurl             0
average_rating      0
location            0
dtype: int64

In [17]:
# clean up area url
def clean_area(area):
    match = re.search('\w\.', area)
    if match:
        area = area.replace(match.group(), '')
        area = area.lstrip()
    else: 
        area = area
    
    return area

In [18]:
data["area"] = data.area.apply(clean_area)

In [288]:
### feature engineering ###

In [19]:
# Parse the climbinfo column
def parse_column(rows):
    list_temp=[]
    for row in rows:
        if not pandas.isnull(row):
            list_temp.append(row.split(','))
        else: 
            list_temp.append('NaN')
            
    return list_temp

list_temp = parse_column(data["climb_info"])

In [20]:
# create a temporary df for storing the climb info
index = range(len(data.index))
columns = ['type', 'toprope','sport', 'trad', 'alpine', 'ice', 'aid', 'bouldering', 'pitches', 'length']

df = pandas.DataFrame(index=index, columns=columns) #data frame consisting for NaN

In [21]:
# fill the data frame with the information parsed from the original climb info column (rewrite this to make it shorter)
count = 0
for row in list_temp:
    
    df.ix[count, 'type'] = list_temp[count][0] #extract first entry as main type    
    
    for item in row:
        
        # determine number of pitches
        match = re.search('.(\d*) pitch', item)
        if match:
            df.ix[count, "pitches"] = int(match.group(1))
        
        # determine type of climb
        if pandas.isnull(df.ix[count,"toprope"]):
            if re.match('TR', item):
                df.ix[count, "toprope"] = 1
            else: 
                df.ix[count, "toprope"] = 0
                
        if pandas.isnull(df.ix[count,"sport"]):
            if re.match('Sport', item):
                df.ix[count, "sport"] = 1
            else: 
                df.ix[count, "sport"] = 0
        
        if pandas.isnull(df.ix[count,"alpine"]):
            if re.match('Alpine', item):
                df.ix[count, "alpine"] = 1
            else: 
                df.ix[count, "alpine"] = 0
       
        if pandas.isnull(df.ix[count,"trad"]):
            if re.match('Trad',item):
                df.ix[count, "trad"] = 1
            else: 
                df.ix[count, "trad"] = 0
        
        if pandas.isnull(df.ix[count,"ice"]):
            if re.match('Ice',item):
                df.ix[count, "ice"] = 1
            else: 
                df.ix[count, "ice"] = 0
    
        if pandas.isnull(df.ix[count,"aid"]):
            if re.match('Aid',item):
                df.ix[count, "aid"] = 1
            else: 
                df.ix[count, "aid"] = 0
        
        if pandas.isnull(df.ix[count,"bouldering"]):
            if re.match('Boulder',item):
                df.ix[count, "bouldering"] = 1
            else: 
                df.ix[count, "bouldering"] = 0
    
             
        # determine length (')
        match = re.search(".(\d*)'", item)
        if match:
            df.ix[count, "length"] = match.group(1)
    
    count = count + 1

In [295]:
df.index = data.index #give df the right index, then join df and data
data = pandas.concat([data, df], axis=1, join='inner')
data.isnull().sum()

name                   0
area                   0
climb_info             0
crag                   0
region                 0
description            0
number_of_votes        0
grade                  0
protection            12
url                    0
areaurl                0
average_rating         0
location               0
type                   0
toprope                0
sport                  0
trad                   0
alpine                 0
ice                    0
aid                    0
bouldering             0
pitches            25464
length             10900
dtype: int64

In [294]:
df.isnull().sum()

type              0
toprope           0
sport             0
trad              0
alpine            0
ice               0
aid               0
bouldering        0
pitches       25464
length        10900
dtype: int64

In [226]:
### climbing grades

In [296]:
def grade_scaling(grades):
    if not pandas.isnull(grades):
        if re.search(re.compile('|'.join(['5.15', '5.14', '5.13','V16','V15','V14','V13','C5','A5', 
                                         'A6', 'M14', 'M13'])), grades): grade_num = 5
        
        elif re.search(re.compile('|'.join(['5.12', '5.11', '5.10','V12','V11','V10','C4','A4', 
                                           'M12', 'M11', 'M10'])), grades): grade_num = 4
        
        elif re.search(re.compile('|'.join(['5.9', '5.8', '5.7', 'V9','V8','V7','C3','A3', 
                                           'M9', 'M8', 'M7'])), grades): grade_num = 3
        
        elif re.search(re.compile('|'.join(['5.6', '5.5', '5.4', 'V6','V5','V4','C2','A2', 
                                           'M6', 'M5', 'M4'])), grades): grade_num = 2
        
        elif re.search(re.compile('|'.join(['5.3', '5.2', '5.1','5.0', 'V3','V2','V1','V0','C1','A1', 
                                           'M3', 'M2', 'M1'])), grades): grade_num = 1
       
        else: grade_num = 0
    else: grade_num = 0
    return grade_num

In [297]:
data["grade_num"] = data.grade.apply(grade_scaling)

In [298]:
### pitches
# for bouldering, assume pitches = 0
data.pitches[data.type == 'Boulder'] = 0
data.pitches[data.pitches.isnull()] = 1

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [299]:
data.isnull().sum()

name                   0
area                   0
climb_info             0
crag                   0
region                 0
description            0
number_of_votes        0
grade                  0
protection            12
url                    0
areaurl                0
average_rating         0
location               0
type                   0
toprope                0
sport                  0
trad                   0
alpine                 0
ice                    0
aid                    0
bouldering             0
pitches                0
length             10900
grade_num              0
dtype: int64

In [300]:
### extract keywords

In [301]:
## doesn't work perfectly yet ... 
def cut_trailingcommas(text):
    if not pandas.isnull(text):
        text = text.split()
        output = text
    
        for element in reversed(text):
            if element == ',':
                output.pop()
    
        return " ".join(output)

    else:
        return text

In [302]:
def remove_overlap(text1, text2):
    match = []
    if not pandas.isnull(text1) and not pandas.isnull(text2):
        try:
            match = re.search(text2,text1)
        except:
            pass
        
        if match:
            try:
                text1 = text1.replace(text2, '')
            except:
                pass
    
    return text1

In [303]:
### clean up the commas in the protection, description, and location columns
data.description = data.description.apply(cut_trailingcommas)
data.location = data.location.apply(cut_trailingcommas)
data.protection = data.protection.apply(cut_trailingcommas)

In [304]:
data['description'] = data.apply(lambda x: remove_overlap(x['description'], x['protection']), axis = 1)
data['description'] = data.apply(lambda x: remove_overlap(x['description'], x['location']), axis = 1)
data['location']    = data.apply(lambda x: remove_overlap(x['location'], x['protection']), axis = 1)

In [235]:
def description_keywords(input_string):
    
    # a list of most frequent "relevant" keywords
    # travers = traverse, stem = stemming, traver = traversing, traverse
    keywords = ['anchor', 'arete', 'awkward', 'block', 'bolt', 'boulder', 'bulge', 'broken', 'canyon',
                'buttress', 'chains', 'chimney', 'classic', 'clean', 'corner', 'crack', 'crimp', 'crux', 
                'difficult','dihedral', 'expos', 'face', 'finger', 'flake', 'gully', 'hand', 'jug', 'ledge', 'lip', 
                'jam','notch', 'moderate', 'ridge', 'vertical', 'horizontal','mantle', 'offwidth',
                'overhang', 'pillar', 'pocket', 'protect', 'ramp','rap', 'roof', 'scramble', 'seam', 'slab',
                'slop', 'solid', 'stance', 'steep', 'stemming', 'sustained',
                'technical', 'tower', 'travers', 'undercling']

    # create a pattern for matching to descriptions
    if not pandas.isnull(input_string) and isinstance(input_string, str):
        pattern = re.compile('|'.join(keywords))
        matches_temp = pattern.findall(input_string)
        matches = set(matches_temp)
    else: matches = np.nan
    return matches

In [305]:
new_col = data.description.apply(description_keywords)

In [308]:
#create a dataframe that contains all the keywords
#def assign_features(input_list): 
    
index = range(len(data.index))
columns = ['anchor', 'arete', 'awkward', 'block', 'bolt', 'boulder', 'bulge', 'broken', 'canyon',
           'buttress', 'chains', 'chimney', 'classic', 'clean', 'corner', 'crack', 'crimp', 'crux', 
           'difficult','dihedral', 'expos', 'face', 'finger', 'flake', 'gully', 'hand', 'jug', 'ledge', 'lip', 
           'jam','notch', 'moderate', 'ridge', 'vertical', 'horizontal','mantle', 'offwidth',
           'overhang', 'pillar', 'pocket', 'protect', 'ramp','rap', 'roof', 'scramble', 'seam', 'slab',
           'slop', 'solid', 'stance', 'steep', 'stemming', 'sustained',
           'technical', 'tower', 'travers', 'undercling']
df = []
df = pandas.DataFrame(0, index=index, columns=columns) #data frame consisting for NaN

count = 0
for row in new_col:
    if not pandas.isnull(row):
        for item in row :
            df.ix[count, item] = 1
            
    count = count + 1

In [310]:
df.rename(columns={'travers': 'traverse', 'crimp': 'crimps', 'expos': 'exposure',
                        'rap': 'rappel', 'slop': 'sloper'}, inplace=True)

In [311]:
df.isnull().sum()

anchor        0
arete         0
awkward       0
block         0
bolt          0
boulder       0
bulge         0
broken        0
canyon        0
buttress      0
chains        0
chimney       0
classic       0
clean         0
corner        0
crack         0
crimps        0
crux          0
difficult     0
dihedral      0
exposure      0
face          0
finger        0
flake         0
gully         0
hand          0
jug           0
ledge         0
lip           0
jam           0
notch         0
moderate      0
ridge         0
vertical      0
horizontal    0
mantle        0
offwidth      0
overhang      0
pillar        0
pocket        0
protect       0
ramp          0
rappel        0
roof          0
scramble      0
seam          0
slab          0
sloper        0
solid         0
stance        0
steep         0
stemming      0
sustained     0
technical     0
tower         0
traverse      0
undercling    0
dtype: int64

In [312]:
df.index = data.index

In [313]:
data = pandas.concat([data, df], axis=1, join='inner')
data.head()
data.keys()

Index([u'name', u'area', u'climb_info', u'crag', u'region', u'description',
       u'number_of_votes', u'grade', u'protection', u'url', u'areaurl',
       u'average_rating', u'location', u'type', u'toprope', u'sport', u'trad',
       u'alpine', u'ice', u'aid', u'bouldering', u'pitches', u'length',
       u'grade_num', u'anchor', u'arete', u'awkward', u'block', u'bolt',
       u'boulder', u'bulge', u'broken', u'canyon', u'buttress', u'chains',
       u'chimney', u'classic', u'clean', u'corner', u'crack', u'crimps',
       u'crux', u'difficult', u'dihedral', u'exposure', u'face', u'finger',
       u'flake', u'gully', u'hand', u'jug', u'ledge', u'lip', u'jam', u'notch',
       u'moderate', u'ridge', u'vertical', u'horizontal', u'mantle',
       u'offwidth', u'overhang', u'pillar', u'pocket', u'protect', u'ramp',
       u'rappel', u'roof', u'scramble', u'seam', u'slab', u'sloper', u'solid',
       u'stance', u'steep', u'stemming', u'sustained', u'technical', u'tower',
       u'traverse', u

In [323]:
#data = data.reset_index()
data.head()

In [325]:
data.to_csv('cleaned_data.csv')

,name,area,climb_info,crag,region,description,number_of_votes,grade,protection,url,...,sloper,solid,stance,steep,stemming,sustained,technical,tower,traverse,undercling
0,mf,the mac wall (something interesting),"Trad, 2 pitches, 180'",the gunks,new york,This is THE standard for 5.9 routes at the Gun...,178,5.9,"Gear to 3"".",http://mountainproject.com/v/mf/105799143,...,0,0,0,0,0,0,0,0,1,0
1,the fastest gun,poke-o-moonshine main face,"Trad, 4 pitches, 500', Grade III",adirondacks,new york,As stated by Don Mellor in his comprehensive g...,49,5.10c,A full rack with doubles in the mid-size range...,http://mountainproject.com/v/the-fastest-gun/1...,...,0,0,1,0,0,0,0,0,0,1
2,modern times,the arrow wall - cck,"Trad, 2 pitches, 250'",the gunks,new york,Modern Times pulls a pretty wild roof on the s...,184,5.9+,Standard Rack.,http://mountainproject.com/v/modern-times/1057...,...,0,0,0,0,0,0,0,0,1,0
3,ants' line,high e,"Trad, 1 pitch, 80'",the gunks,new york,A great line with good protection within safe ...,281,5.9,"A standard light rack of nuts and cams to 3"".",http://mountainproject.com/v/ants-line/105803260,...,0,0,1,1,0,0,0,0,0,0
4,directissima,high e,"Trad, 2 pitches, 120'",the gunks,new york,Directissima is most commonly (if not always) ...,216,5.9+,"Standard Rack, many fixed pins on the climb.",http://mountainproject.com/v/directissima/1057...,...,0,0,0,0,0,0,0,0,1,1
